### Prepare environment

In [1]:
### Prepare environment
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}:")
# set proxy
_set_env("PROXY_VALUE")
proxy = os.getenv("PROXY_VALUE")
for proxy_var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    os.environ[proxy_var] = proxy
# GOOGLE API KEY
_set_env("GOOGLE_API_KEY")
print("proxy_value:", proxy)

proxy_value: http://localhost:7897


### Set langsmith

In [2]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

### Agent
In the context of agents, waiting for user feedback is especially useful for asking clarifying qeustions.
To illustrate this, we'll create a simple ReAct-style agent capable of tool calling.

In [8]:
# Set up the state
from langgraph.graph import MessagesState, START

# Set up the tool
# We will have one real tool -a search tool
# We'll also have one "fake" tool -a "ask_human" tool
# Here we define any ACTUAL tools
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

@tool
def search(query: str):
    """
    Call to surf the web.
    """
    # This is a placehodler for the acutal implementation
    # Don't let the LLM know this though
    return f"I looked up: {query}. Result: It's sunny in Chengdu, but you better look out if your're a Gemini."


tools = [search]
tool_node = ToolNode(tools)

# Set up the Gemini model
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.2,
    max_tokens=512,
    timeout=None,
    max_retries=2
)

from pydantic import BaseModel

# We are going "bind" call tools to the model
# We have the ACTUAL tools from above, but we also need a mock tool to ask a human
# Since `bind_tools` takes in tools but also just tool definitions
# We can define a tool definition for `ask_human`
class AskHuman(BaseModel):
    """Ask the human a question"""
    question: str

model = model.bind_tools(tools + [AskHuman])

# Define node and conditonal edges

# Define the function that determines whether to contine or not
def should_contine(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END

    # If tool call is asking Human, we return that node
    # You could also add logic here to let some system know that there's something that requires Human input
    # For example, send a slack message, etc
    elif last_message.tool_calls[0]["name"] == "AskHuman":
        return "ask_human"
    # Otherwise if there is, we contine
    else:
        return "action"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}
    


